<a href="https://colab.research.google.com/github/zhenwenzhang/BERT-SLU/blob/master/BERT_for_intent_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/zhenwenzhang/BERT-SLU.git
# % cd BERT-SLU
# ! rm -rf log
# ! git

Cloning into 'BERT-SLU'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 64 (delta 27), reused 15 (delta 4), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [2]:
"""BERT finetuning runner."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import csv
import os
from bert import modeling
from bert import optimization
from bert import tokenization
from bert import dataloader
import tensorflow as tf
from sklearn import metrics

W0725 13:05:37.254161 139888097109888 deprecation_wrapper.py:119] From /content/BERT-SLU/bert/optimization.py:84: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# download pre-trained models
# ! wget -P checkpoints https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
# ! wget -P checkpoints https://storage.googleapis.com/bert_models/2018_11_03/multilingual_L-12_H-768_A-12.zip
# ! wget -P checkpoints https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip 
# ! wget -P checkpoints https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
# ! wget -P checkpoints https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
# ! wget -P checkpoints https://storage.googleapis.com/bert_models/2018_10_18/cased_L-24_H-1024_A-16.zip

# ! unzip -o checkpoints/multi_cased_L-12_H-768_A-12.zip -d checkpoints
# ! unzip -o checkpoints/multilingual_L-12_H-768_A-12.zip -d checkpoints
# ! unzip -o checkpoints/uncased_L-12_H-768_A-12.zip -d checkpoints
# ! unzip -o checkpoints/uncased_L-24_H-1024_A-16.zip -d checkpoints
# ! unzip -o checkpoints/cased_L-12_H-768_A-12.zip -d checkpoints
# ! unzip -o checkpoints/cased_L-24_H-1024_A-16.zip -d checkpoints

# ! rm -rf *.zip

### Parameter Settings

In [0]:
max_seq_length = 50 #@param {type:"integer"}
train_batch_size = 32 #@param {type:"integer"}
eval_batch_size = 32 #@param {type:"integer"}
predict_batch_size = 32 #@param {type:"integer"}

warmup_proportion = 0.1
save_checkpoints_steps = 1000
log_step_count_steps = 10
save_summary_steps = 1

learning_rate = 5e-5 #@param ["5e-5", "3e-5", "2e-5"] {type:"raw"}
num_train_epochs = 1 #@param {type:"integer",min:1, max:10, step:1}
do_train = True #@param ["False", "True"] {type:"raw"}
do_eval = True #@param ["False", "True"] {type:"raw"}
do_predict = True #@param ["False", "True"] {type:"raw"}

log_dir = 'log' #@param {type:"string"}
data_dir = 'data/atis' #@param ["data/atis", "data/snips"]
checkpoints = 'checkpoints/multi_cased_L-12_H-768_A-12' #@param ["checkpoints/multi_cased_L-12_H-768_A-12","checkpoints/cased_L-12_H-768_A-12","checkpoints/cased_L-24_H-1024_A-16","checkpoints/uncased_L-12_H-768_A-12","checkpoints/uncased_L-24_H-1024_A-16"]
bert_config_file = os.path.join(checkpoints, 'bert_config.json')
vocab_file = os.path.join(checkpoints, 'vocab.txt')
init_checkpoint = os.path.join(checkpoints, 'bert_model.ckpt')

if checkpoints.split('/')[1].startswith('u'):
  do_lower_case = True
else:
  do_lower_case = False    

### Model Function

In [0]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels):
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids)

    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    if is_training:
        # I.e., 0.1 dropout
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, logits, probabilities)

In [0]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate, num_train_steps, num_warmup_steps):
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_real_example = None
        if "is_real_example" in features:
            is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
            is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels)

        predicted_class = tf.argmax(logits, axis=-1, output_type=tf.int32)

        accuracy = tf.metrics.accuracy(labels=label_ids,
                                       predictions=predicted_class,
                                       weights=is_real_example, name="acc_op")
        tf.summary.scalar("accuracy", accuracy[1])

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)

            tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

#         tf.logging.info("**** Trainable Variables ****")
#         for var in tvars:
#             init_string = ""
#             if var.name in initialized_variable_names:
#                 init_string = ", *INIT_FROM_CKPT*"
#             tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape, init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op
            )
        elif mode == tf.estimator.ModeKeys.EVAL:

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops={"accuracy": accuracy}
            )
        else:
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"predicted_class": predicted_class,
                             "true_class": label_ids}
            )
        return output_spec

    return model_fn

### Train

In [15]:
tf.logging.set_verbosity(tf.logging.INFO)
if not do_train and not do_eval and not do_predict:
    raise ValueError(
        "At least one of `do_train`, `do_eval` or `do_predict' must be True.")
tf.gfile.MakeDirs(log_dir)
processor = DataProcessor(data_dir)
label_map, num_labels = processor.get_labels_info()
tokenization.validate_case_matches_checkpoint(do_lower_case, init_checkpoint)
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

if max_seq_length > bert_config.max_position_embeddings:
    raise ValueError("Cannot use sequence length %d because the BERT model "
                     "was only trained up to sequence length %d" %
                     (max_seq_length, bert_config.max_position_embeddings))
tokenizer = tokenization.FullTokenizer(
    vocab_file=vocab_file, do_lower_case=do_lower_case)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
run_config = tf.estimator.RunConfig(
    model_dir=log_dir,
    session_config=config,
    save_checkpoints_steps=save_checkpoints_steps,
    log_step_count_steps=log_step_count_steps,
    save_summary_steps=save_summary_steps)
train_examples = None
num_train_steps = None
num_warmup_steps = None

if do_train:
    train_examples = processor.get_train_examples()
    num_train_steps = int(
        len(train_examples) / train_batch_size * num_train_epochs)
    num_warmup_steps = int(num_train_steps * warmup_proportion)

model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=num_labels,
    init_checkpoint=init_checkpoint,
    learning_rate=learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(model_fn=model_fn, config=run_config)

# Training
if do_train:
    train_file = os.path.join(log_dir, "train.tf_record")
    file_based_convert_examples_to_features(
        train_examples, label_map, max_seq_length, tokenizer, train_file)
    tf.logging.info("***** Running training *****")
    tf.logging.info("  Num examples = %d", len(train_examples))
    tf.logging.info("  Batch size = %d", train_batch_size)
    tf.logging.info("  Num steps = %d", num_train_steps)
    train_input_fn = file_based_input_fn_builder(
        input_file=train_file,
        seq_length=max_seq_length,
        is_training=True,
        drop_remainder=False,
        batch_size=train_batch_size)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

W0725 13:05:38.547155 139888097109888 deprecation_wrapper.py:119] From /content/BERT-SLU/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I0725 13:05:39.034579 139888097109888 estimator.py:209] Using config: {'_model_dir': 'log', '_tf_random_seed': None, '_save_summary_steps': 1, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  allow_growth: true
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f39cae48940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_re

### Evaluate

In [16]:
if do_eval:
    eval_examples = processor.get_dev_examples()
    num_actual_eval_examples = len(eval_examples)
    eval_file = os.path.join(log_dir, "eval.tf_record")
    file_based_convert_examples_to_features(
        eval_examples, label_map, max_seq_length, tokenizer, eval_file)
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                    len(eval_examples), num_actual_eval_examples,
                    len(eval_examples) - num_actual_eval_examples)
    tf.logging.info("  Batch size = %d", eval_batch_size)

    eval_input_fn = file_based_input_fn_builder(
        input_file=eval_file,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=False,
        batch_size=eval_batch_size)

    result = estimator.evaluate(input_fn=eval_input_fn)

    output_eval_file = os.path.join(log_dir, "eval_results.txt")
    with tf.gfile.GFile(output_eval_file, "w") as writer:
        tf.logging.info("***** Eval results *****")
        for key in sorted(result.keys()):
            tf.logging.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

I0725 13:08:11.726988 139888097109888 <ipython-input-10-4818e34a2ae0>:9] Writing example 0 of 500
I0725 13:08:11.728673 139888097109888 <ipython-input-9-319d0dfaaa64>:63] *** Example ***
I0725 13:08:11.729657 139888097109888 <ipython-input-9-319d0dfaaa64>:64] guid: dev-0
I0725 13:08:11.731560 139888097109888 <ipython-input-9-319d0dfaaa64>:66] tokens: [CLS] i want to fly from bost ##on at 838 am and arrive in den ##ver at 1110 in the morning [SEP]
I0725 13:08:11.733252 139888097109888 <ipython-input-9-319d0dfaaa64>:68] input_ids: 101 177 21528 10114 26155 10188 29495 10263 10160 82665 10392 10111 27814 10106 10140 12563 10160 106270 10106 10105 28757 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I0725 13:08:11.735260 139888097109888 <ipython-input-9-319d0dfaaa64>:70] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I0725 13:08:11.737380 139888097109888 <ipython-input-9-319d0dfaaa64>:72] segment_ids: 0 0 0 0 0 0

### Predict

In [19]:
if do_predict:

    predict_examples = processor.get_test_examples()
    num_actual_predict_examples = len(predict_examples)

    predict_file = os.path.join(log_dir, "predict.tf_record")
    if not tf.gfile.Exists(predict_file):
        file_based_convert_examples_to_features(predict_examples, label_map,
                                                max_seq_length, tokenizer,
                                                predict_file)

    tf.logging.info("***** Running prediction*****")
    tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                    len(predict_examples), num_actual_predict_examples,
                    len(predict_examples) - num_actual_predict_examples)
    tf.logging.info("  Batch size = %d", predict_batch_size)

    predict_input_fn = file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=False,
        batch_size=predict_batch_size)

    result = estimator.predict(input_fn=predict_input_fn)
    label_map_new = {v: k for k, v in label_map.items()}


    output_predict_file = os.path.join(log_dir, "test_results.tsv")
    with tf.gfile.GFile(output_predict_file, "w") as writer:
        writer.write("line, true, predict \n")
        tf.logging.info("***** Predict results *****")

        true_classes = []
        predicted_classes = []

        for i, item in enumerate(result):
                true_class = item["true_class"]
                predicted_class = item["predicted_class"]

                true_classes.append(true_class)
                predicted_classes.append(predicted_class)

                if predicted_class != true_class:
                    output_line = "{}, {}, {}\n".format(i + 1, label_map_new[true_class],
                                                        label_map_new[predicted_class])
                    writer.write(output_line)
    accuracy = metrics.accuracy_score(true_classes, predicted_classes)
    classification_report = metrics.classification_report(true_classes, predicted_classes)
    tf.logging.info("Accuracy: %s", accuracy)
    tf.logging.info("\nAccuracy: %s", classification_report)

I0725 13:31:20.848347 139888097109888 <ipython-input-19-d27651733b67>:12] ***** Running prediction*****
I0725 13:31:20.851618 139888097109888 <ipython-input-19-d27651733b67>:15]   Num examples = 893 (893 actual, 0 padding)
I0725 13:31:20.852965 139888097109888 <ipython-input-19-d27651733b67>:16]   Batch size = 32
I0725 13:31:20.856503 139888097109888 <ipython-input-19-d27651733b67>:32] ***** Predict results *****
I0725 13:31:20.900712 139888097109888 estimator.py:1145] Calling model_fn.
I0725 13:31:20.901857 139888097109888 <ipython-input-14-64f48bfe5f20>:4] *** Features ***
I0725 13:31:20.905015 139888097109888 <ipython-input-14-64f48bfe5f20>:6]   name = input_ids, shape = (?, 50)
I0725 13:31:20.907747 139888097109888 <ipython-input-14-64f48bfe5f20>:6]   name = input_mask, shape = (?, 50)
I0725 13:31:20.912290 139888097109888 <ipython-input-14-64f48bfe5f20>:6]   name = is_real_example, shape = (?,)
I0725 13:31:20.913725 139888097109888 <ipython-input-14-64f48bfe5f20>:6]   name = label